# Machine Learning project CS-433: NMR spectroscopy supervised learning



## Schedules:

* Week 10 (18-24 November): 
 * Tests of various linear models/simple NN on a 10% subset of data
* Week 11 (25-1 December):
 * Feature selection: being able to come with a good set of features
* Week 12 (2-8 December):
 * Start of big scale analysis with Spark, implementation of the models which perform well at small scale
* Week 13 (9-15 December):
 * Wrapping up
* Week 14 (16-22 December): 
 * 19th December: Deadline

## Table of contents

1. [Log Book](#log)
2. [Pipeline](#pipeline)
3. [Data Processing](#data_proc) <br>
&emsp;3.1. [Data Vizualisation](#data_viz) <br>
&emsp;3.2 [Outliers detection](#outliers) <br>
  &emsp;&emsp;3.2.1 [DBSCAN](#dbscan) <br>
  &emsp;&emsp;3.2.2 [Inter quantile range method](#iqr) <br>
&emsp;3.3 [Scaling](#scaling) <br>
&emsp;3.4 [Dimensionality reduction](#dim_red) <br>
  &emsp;&emsp;3.4.1 [PCA](#pca) <br>
&emsp;3.5 [Models](#models) <br>
  &emsp;&emsp;3.5.1 [Linear Models](#lin_mods) <br>
  &emsp;&emsp;3.5.2 [Neural Networks](#NN) <br>
4. [Main](#main) <br>
   4.1 [ANN implementation](#ann_imp) <br>
    

In [ ]:
import os
import re
import pickle
import scipy.stats
import sklearn.metrics

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import norm
from itertools import combinations

from IPython.core.debugger import set_trace


from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

In [ ]:
# For neural net part
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

## 1. Log Book
<a id='log'></a>


We write here the log of the different techniques/improvements we add to the program: **cf log/models_log.txt** for the different models already tested and their results.

## 2. Pipeline
<a id='pipeline'></a>

In [ ]:
#pipeline graph coming soon

## 3. Data Processing
<a id = 'data_proc'></a>

In [ ]:
tot_data_X = np.load('data/CSD-10k_H_fps_1k_MD_n_12_l_9_rc_3.0_gw_0.3_rsr_1.0_rss_2.5_rse_5.npy',mmap_mode='r')
tot_data_y = np.load('data/CSD-10k_H_chemical_shieldings.npy',mmap_mode='r')
DATA_LEN = tot_data_X.shape[0]
DATA_COLS = tot_data_X.shape[1]

In [ ]:
def load_data(n_samples,tot_data_x = tot_data_X,tot_data_y = tot_data_y):
    #np.random.seed(14)
    mask_data = np.random.permutation(DATA_LEN)[:n_samples]

    data_X = tot_data_X[mask_data]
    data_y = tot_data_y[mask_data]
    return data_X, data_y

In [ ]:
def load_data_train_test(n_samples,tot_data_x = tot_data_X,tot_data_y = tot_data_y):
    data_X, data_y = load_data(n_samples,tot_data_x,tot_data_y)
    X_train,X_test,y_train,y_test = train_test_split(data_X,data_y,test_size = 0.2)
    return X_train,X_test,y_train,y_test

### 3.1 Data Vizualisation
<a id='data_viz'></a>

In [ ]:
data_X,data_y = load_data(3000)
data_X_df = pd.DataFrame(data_X)
data_y_df = pd.DataFrame(data_y)

In [ ]:
mask = np.random.permutation(DATA_COLS)[:9]
fig, axes = plt.subplots(nrows=3, ncols=3)
fig.set_size_inches(11,11)
for ind,i in enumerate(mask):
    index = np.unravel_index(ind,(3,3))
    axes[index].ticklabel_format(style='sci',scilimits=(-3,4),axis='both')
    data_X_df.iloc[:,i].hist(ax = axes[index],bins = 50)
    #data_X_df.iloc[:,i].plot.box(ax = axes[index])

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3)
fig.set_size_inches(11,11)
for ind,i in enumerate(mask):
    index = np.unravel_index(ind,(3,3))
    axes[index].ticklabel_format(style='sci',scilimits=(-3,4),axis='both')
    #data_X_df.iloc[:,i].hist(ax = axes[index],bins = 50)
    data_X_df.iloc[:,i].plot.box(ax = axes[index])

We see that the different features are scaled pretty differently, we might want to scale them beforehand. Since they don' look like following a gaussian, we'll apply min/max scaling: but in order to do so, we first need to get rid of the outliers thanks to one of the following methods
* Zscore: not adapted as our data might not be gaussian
* DBScan:
* Isolation Forest:

### 3.2 Outliers detection
<a id='outliers'></a>

In [ ]:
def filter_outliers(meth,X_train,y_train):
    """
    Drops the outliers values from the dataset
    masks: [[int]]: each array's indexes correspond to the samples that the corresponding feature considers as outliers.
    """
    masks = np.array([meth(feat) for feat in X_train.T])
    masks = np.hstack(masks)
   # set_trace()
    X_train = np.delete(X_train,masks,axis = 0)
    y_train = np.delete(y_train,masks,axis = 0)
    return X_train,y_train

#### 3.2.1 DBSCAN
<a id = 'dbscan'></a>

Problem: computationally too demanding.

In [ ]:
#clustering = DBSCAN(eps=0.3, min_samples=2).fit(X_train)
#with np.printoptions(threshold=np.inf):
#    print(clustering.labels_)

#### 3.2.2 Interquartile range method (IQR)
<a id = 'iqr'></a>

Consists in considering as outliers all data points that lie in >1.5 interquartile range from the quartiles.

In [ ]:
def IQR(ys):
    """
    returns the array of indexes of the samples considered as outliers according to IQR"""
    q1, q3 = np.percentile(ys, [5, 95])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))[0]

### 3.3 Scaling
<a id='scaling'></a>

#### 3.3.1 Min/max Scaling
<a id='minmax'></a>

In [ ]:
def apply_scaler(scaler,X_train,X_test):
    X_train = minmx_scaler.fit_transform(X_train)
    X_test = minmx_scaler.transform(X_test)
    return X_train,X_test

### 3.4 Dimensionality reduction
<a id='dim_red'></a>

#### 3.4.1 PCA
<a id = 'pca'></a>

In [ ]:
def plot_PCA(n_comp,X_train):
    """
    displays the 'elbow' of the PCA, ie the screeplot"""
    pca = PCA(n_components = n_comp)
    pca.fit(X_train)
    plt.figure(1)
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Number of components')
    plt.ylabel('Cumulative explained variance')
    plt.show()
    
X_train,_,_,_ = load_data_train_test(3000)
plot_PCA(70,X_train)

In [ ]:
def do_PCA(X_train,X_test,n):
    pca = PCA(n_components = n)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    return X_train,X_test

### 3.5 Models
<a id='models'></a>

#### 3.5.1 Linear Models
<a id='lin_mods'></a>

In [ ]:
def test_lin_model(reg, X_train,y_train,X_test,y_test):
    """
    Test a model reg which is expected to be already instantiated
    return score_train,score_test"""
    reg.fit(X_train,y_train)
    train_R2 = reg.score(X_train,y_train)
    test_R2 = reg.score(X_test,y_test)
    y_hat = reg.predict(X_test)
    mse = mean_squared_error(y_test,y_hat)
    mae = mean_absolute_error(y_test,y_hat)
    print("Obtained score on train set %2.2f " % train_R2)
    print("Obtained score on test set %2.2f " % test_R2)
    print("Obtained MSE on test set %2.2f " % mse)
    print("Obtained MAE on test set %2.2f " % mae)
    return mse, mae

#### 3.5.2 Neural Nets

<a id='NN'></a>

In [ ]:
class Net(nn.Module):
    def __init__(self, n):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n,100)
        #self.fc2 = nn.Linear(80,50)
        #self.fc3 = nn.Linear(624,624)
        self.fc4 = nn.Linear(100,1)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x

In [ ]:
def train_model(model, train_input, train_target, mini_batch_size, monitor_loss=False):
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr = 1e-3) 
    nb_epochs = 100
    
    # Monitor loss
    losses = []
    
    for e in range(nb_epochs):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            
            sum_loss += loss.item() # compute loss for each mini batch for 1 epoch
            
            optimizer.step()
        
        # Monitor loss
        losses.append(sum_loss)
        
        print('[epoch {:d}] loss: {:0.2f}'.format(e+1, sum_loss))
    
    if monitor_loss:
        return losses

In [ ]:
def make_pred(model, data_input):
    y_hat = model(data_input)
    return y_hat

In [ ]:
def compute_score(y_actual, y_pred):
    mse = mean_squared_error(y_actual, y_pred)
    mae = mean_absolute_error(y_actual, y_pred)
    print("Obtained MSE on test set %2.2f " % mse)
    print("Obtained MAE on test set %2.2f " % mae)
    return mse, mae

## 4. Main
<a id='main'></a>

Each cell here is meant to do a whole pipeline, from loading a certain number of samples, preprocessing etc. We keep using the R2 score as our metric

In [ ]:
X_train_save,X_test_save,y_train_save,y_test_save = load_data_train_test(10000)

In [ ]:
#Linear Regression
X_train = np.copy(X_train_save)
X_test = np.copy(X_test_save)
y_train = np.copy(y_train_save) # y_train?
y_test = np.copy(y_test_save)
X_train,y_train = filter_outliers(IQR,X_train,y_train)
X_train,X_test = do_PCA(X_train,X_test,40)
minmx_scaler = MinMaxScaler()
X_train, X_test = apply_scaler(minmx_scaler,X_train,X_test)
lin = LinearRegression(fit_intercept = True).fit(X_train,y_train)
train_score,test_score = test_lin_model(lin,X_train,y_train,X_test,y_test)

In [ ]:
#Ridge regression
X_train = np.copy(X_train_save)
X_test = np.copy(X_test_save)
y_train = np.copy(y_train_save)
y_test = np.copy(y_test_save)
do_PCA(X_train,X_test,40)
minmx_scaler = MinMaxScaler()
X_train, X_test = apply_scaler(minmx_scaler,X_train,X_test)
rid = Ridge()#.fit(X_train,y_train)
train_score,test_score = test_lin_model(rid,X_train,y_train,X_test,y_test)

In [ ]:
#Kernel ridge
X_train = np.copy(X_train_save)
X_test = np.copy(X_test_save)
y_train = np.copy(y_train_save)
y_test = np.copy(y_test_save)
X_train, X_test = do_PCA(X_train,X_test,40)
minmx_scaler = MinMaxScaler()
X_train, X_test = apply_scaler(minmx_scaler,X_train,X_test)

kr = KernelRidge(alpha=0.1) #.fit(X_train,y_train)
train_score,test_score = test_lin_model(kr, X_train, y_train, X_test, y_test)

In [ ]:
#Neural nets
#Get the data
X_train = np.copy(X_train_save)
X_test = np.copy(X_test_save)
y_train = np.copy(y_train_save)
y_test = np.copy(y_test_save)
X_train, X_test = do_PCA(X_train,X_test,40)
minmx_scaler = MinMaxScaler()
X_train, X_test = apply_scaler(minmx_scaler,X_train,X_test)

#Convert to tensors
train_input = torch.Tensor(X_train)
test_input = torch.Tensor(X_test)
train_target = torch.Tensor(y_train.reshape(len(y_train), 1))
test_target = torch.Tensor(y_test.reshape(len(y_test), 1))

In [ ]:
# Sanity check
print(train_input.shape)
print(train_target.shape)
print(test_input.shape)
print(test_target.shape)

In [ ]:
#Train model
mini_batch_size = 10
model = Net(40)
losses = train_model(model, train_input, train_target, mini_batch_size, monitor_loss=True)

In [ ]:
#Make predictions
y_hat = make_pred(model, test_input)

#Compute score
mse_nn, mae_nn = compute_score(y_test, y_hat.detach().numpy())

In [ ]:
plt.plot(np.arange(100)+1, losses)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Loss evolution during training', fontsize=15)

In [ ]:
#Random forest regressor
X_train = np.copy(X_train_save)
X_test = np.copy(X_test_save)
y_train = np.copy(y_train_save)
y_test = np.copy(y_test_save)
X_train, X_test = do_PCA(X_train,X_test,40)
minmx_scaler = MinMaxScaler()
X_train, X_test = apply_scaler(minmx_scaler,X_train,X_test)

rf = RandomForestRegressor(max_depth=15, random_state=0, n_estimators=150)
train_score,test_score = test_lin_model(rf, X_train, y_train, X_test, y_test)

In [ ]:
plt.bar(np.arange(40)+1, rf.feature_importances_)
plt.xlabel('Feature number')
plt.ylabel('')
plt.title('Feature importance')

#### 4.1 ANN implementation
<a id='ann_imp'></a>

In [ ]:
# architecture 1
mlp_1 = Sequential()

mlp_1.add(Dense(100, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))
mlp_1.add(Dense(1, kernel_initializer='normal',activation='linear'))

mlp_1.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
mlp_1.summary()

In [ ]:
# architecture 2
mlp_2 = Sequential()

# The Input Layer
mlp_2.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers
mlp_2.add(Dense(256, kernel_initializer='normal',activation='relu'))
mlp_2.add(Dense(256, kernel_initializer='normal',activation='relu'))
mlp_2.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer
mlp_2.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network
mlp_2.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
mlp_2.summary()

In [ ]:
#checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
#checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
#callbacks_list = [checkpoint]

In [ ]:
ann_model = mlp_2
ann_model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split = 0.2) #callbacks=callbacks_list)

In [ ]:
#Test model
y_hat = ann_model.predict(X_test)

#Compute score
mse_nn, mae_nn = compute_score(y_test, y_hat)